# below codes are obsolete

### Function for plotting by time (50Hz)

In [ ]:

def plot_movement(file_path, sample_rate=50):
    df = pd.read_csv('./Motion/'+file_path)
    time = np.linspace(0, len(df)/sample_rate, len(df))

    fig, axs = plt.subplots(3, 4, figsize=(20, 15))  # 3 rows for x, y, z axes and 4 columns for the four graphs
    
    for i, axis in enumerate(['x', 'y', 'z']):
        acceleration = df[f'userAcceleration.{axis}']
        velocity = cumtrapz(acceleration, dx=1/sample_rate, initial=0)
        position = cumtrapz(velocity, dx=1/sample_rate, initial=0)
        sp = np.fft.fft(acceleration)
        freq = np.fft.fftfreq(len(sp), 1 / sample_rate)
        
        data = [acceleration, velocity, position, sp]  # List of data for the four graphs
        titles = ['Acceleration', 'Velocity', 'Position', 'Frequency Analysis']
        ylabels = ['Acceleration (m/s^2)', 'Velocity (m/s)', 'Position (m)', 'Amplitude']
        
        for j in range(4):
            ax = axs[i, j]
            if j < 3:  # For Acceleration, Velocity, and Position
                ax.plot(time, data[j], label=f'{titles[j]} ({axis.upper()}-axis)')
                ax.set_xlabel('Time (s)')
            else:  # For Frequency Analysis
                ax.plot(freq[freq > 0], np.abs(data[j][freq > 0]), label=f'FFT ({axis.upper()}-axis)')
                ax.set_xlabel('Frequency (Hz)')
            ax.set_title(f'{titles[j]} ({axis.upper()}-axis)')
            ax.set_ylabel(ylabels[j])
            ax.legend()
    
    #plt.tight_layout()
    plt.show()

plot_movement('jog_9/sub_7.csv')

### function for plotting with Acc + Gravity 

In [ ]:
def plot_movement_grav(file_path, sample_rate=50):
    df = pd.read_csv('./Motion/'+file_path)
    df['label'] = file_path[:3]
    time = np.linspace(0, len(df)/sample_rate, len(df))

    fig, axs = plt.subplots(3, 4, figsize=(20, 15))  # 3 rows for x, y, z axes and 4 columns for the four graphs
    
    # title coming from the file_path format (jog_16/sub_7.csv)
    activity = file_path.split('/')[0].upper()
    person = file_path.split('/')[-1].upper()[:5]
    full_title = f"{activity} ({person})"
    fig.suptitle(full_title,fontsize=40)
    
    for i, axis in enumerate(['x', 'y', 'z']):
        df[f'Acceleration.{axis}'] = df[f'gravity.{axis}']+df[f'userAcceleration.{axis}']
        # acceleration = df[f'Acceleration.{axis}']
        acceleration = highpass_filter(df[f'Acceleration.{axis}'], cutoff=0.1, fs=50)
        velocity = cumtrapz(acceleration, dx=1/sample_rate, initial=0)
        position = cumtrapz(velocity, dx=1/sample_rate, initial=0)
        sp = np.fft.fft(acceleration)
        freq = np.fft.fftfreq(len(sp), 1 / sample_rate)
        
        data = [acceleration, velocity, position, sp]  # List of data for the four graphs
        titles = ['Acceleration', 'Velocity', 'Position', 'Frequency Analysis']
        ylabels = ['Acceleration (m/s^2)', 'Velocity (m/s)', 'Position (m)', 'Amplitude']
        
        for j in range(4):
            ax = axs[i, j]
            if j < 3:  # For Acceleration, Velocity, and Position
                ax.plot(time, data[j], label=f'{titles[j]} ({axis.upper()}-axis)')
                ax.set_xlabel('Time (s)')
            else:  # For Frequency Analysis
                ax.plot(freq[freq > 0], np.abs(data[j][freq > 0]), label=f'FFT ({axis.upper()}-axis)')
                ax.set_xlabel('Frequency (Hz)')
            ax.set_title(f'{titles[j]} ({axis.upper()}-axis)')
            ax.set_ylabel(ylabels[j])
            ax.legend()
    
    #plt.tight_layout()
    plt.show()

# Run the function on the dataset
plot_movement_grav('jog_9/sub_7.csv')

### Additional lines to annotate dominant frequency and amplitude


In [ ]:
...
else:  # For Frequency Analysis
    ax.plot(freq[freq > 0], np.abs(data[j][freq > 0]), label=f'FFT ({axis.upper()}-axis)')
    ax.set_xlabel('Frequency (Hz)')
    
    # Additional lines to annotate dominant frequency and amplitude
    pos_freq = freq[freq > 0]
    pos_sp = np.abs(sp[freq > 0])
    dominant_frequency = pos_freq[np.argmax(pos_sp)]
    amplitude_at_dominant_frequency = np.max(pos_sp)
    ax.annotate(f'Dominant Frequency: {dominant_frequency:.2f} Hz\nAmplitude: {amplitude_at_dominant_frequency:.2f}', 
                xy=(dominant_frequency, amplitude_at_dominant_frequency), 
                xytext=(dominant_frequency + 1, amplitude_at_dominant_frequency),
                arrowprops=dict(facecolor='red', arrowstyle='->'),
                fontsize=12,
                color='red')
...


In [ ]:
#nested for loop
for i, axis in enumerate(['x', 'y', 'z']):
    for j in range(4):
        print('Row '+str(i),'Axis '+axis,'Col(Acc, Vel, Pos, Freq) '+str(j))

## FFT feature extraction

In [ ]:
def extract_fft_features(file_path, sample_rate=50):
    df = pd.read_csv('./Motion/'+file_path)
    activity_label = file_path.split('/')[0]
    person_id = file_path.split('/')[-1].split('_')[1].split('.')[0]

    fft_features = {}
    fft_features['activity'] = activity_label
    fft_features['person_id'] = person_id

    for axis in ['x', 'y', 'z']:
        df[f'Acceleration.{axis}'] = df[f'gravity.{axis}'] + df[f'userAcceleration.{axis}']
        acceleration = highpass_filter(df[f'Acceleration.{axis}'], cutoff=0.1, fs=50)

        sp = np.fft.fft(acceleration)
        freq = np.fft.fftfreq(len(sp), 1 / sample_rate)

        # Assuming the FFT is symmetrical, only consider the positive part
        pos_freq = freq[freq > 0]
        pos_sp = np.abs(sp[freq > 0])

        # Get the dominant frequency and its amplitude
        dominant_frequency = pos_freq[np.argmax(pos_sp)]
        amplitude_at_dominant_frequency = np.max(pos_sp)

        fft_features[f'dominant_frequency_{axis}'] = dominant_frequency
        fft_features[f'amplitude_at_dominant_frequency_{axis}'] = amplitude_at_dominant_frequency

    return fft_features

# Example usage:
file_path = 'jog_9/sub_7.csv'
fft_features = extract_fft_features(file_path)
print(fft_features)


## ------save--------

## ----Code Starts from here----

Total Acceleration=Gravity+User Acceleration

Velocity: We'll integrate the acceleration data to obtain velocity. For the integration, we'll need to know the time interval between each data point. Could you please provide this information?

Position: After getting the velocity, we'll integrate again to obtain the position.

Energy: Kinetic energy is given by E = 1/2mv2

Dominant Frequencies: Fast Fourier Transform (FFT) on the acceleration data to find the dominant frequencies.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.fftpack import fft

#for png file save
import os

#for ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Accelerometer integration showed huge drifts 
### How can I correct this??
- Trick 1 : Sum Gravity + userAcceleration
- Trick 2 : Apply High pass filter

In [ ]:
# High pass filter

import numpy as np
from scipy.signal import butter, filtfilt

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

### I want to do frequency analysis, and adding annotation in my frequency plots

In [ ]:
# High pass filter
# Acc = (Gravity + UserAcc)
# Annotation for dominent frequency
# Add dotted line for average velocity 
# Add annotation for max velocity moment (just for fun)

def plot_movement_grav_HF_anno(file_path, folder_to_save, sample_rate=50):
    df = pd.read_csv('./Motion/'+file_path)
    df['label'] = file_path[:3]
    time = np.linspace(0, len(df)/sample_rate, len(df))

    fig, axs = plt.subplots(3, 4, figsize=(20, 15))  # 3 rows for x, y, z axes and 4 columns for the four graphs
    
    # title coming from the file_path format (jog_16/sub_7.csv)
    activity = file_path.split('/')[0].upper()
    person = file_path.split('/')[-1][4:].replace('.csv','')
    full_title = f"{activity} (#{person})"
    fig.suptitle(full_title,fontsize=40)
    
    for i, axis in enumerate(['x', 'y', 'z']):
        df[f'Acceleration.{axis}'] = df[f'gravity.{axis}']+df[f'userAcceleration.{axis}']
        
        # acceleration = df[f'Acceleration.{axis}']
        acceleration = highpass_filter(df[f'Acceleration.{axis}'], cutoff=0.1, fs=50)
        velocity = cumtrapz(acceleration, dx=1/sample_rate, initial=0)
        position = cumtrapz(velocity, dx=1/sample_rate, initial=0)
        sp = np.fft.fft(acceleration)
        freq = np.fft.fftfreq(len(sp), 1 / sample_rate)
        
        data = [acceleration, velocity, position, sp]  # List of data for the four graphs
        titles = ['Acceleration', 'Velocity', 'Position', 'Frequency Analysis']
        ylabels = ['Acceleration (m/s^2)', 'Velocity (m/s)', 'Position (m)', 'Amplitude']
        
        for j in range(4):
            ax = axs[i, j]
            if j < 3:  # For Acceleration, Velocity, and Position
                ax.plot(time, data[j], label=f'{titles[j]} ({axis.upper()}-axis)')
                ax.set_xlabel('Time (s)')
                ax.legend()
                # Add average line for velocity plot
                if titles[j] == 'Velocity':
                    avg_velocity = np.mean(data[j])
                    ax.axhline(y = avg_velocity, color = 'r', linestyle='--',label =f'Average Velocity = {avg_velocity:.2f}')
                
                    # Add annotation for max velocity moment (just for fun)
                    # Find max velocity and its corresponding time
                    max_velocity = np.max(data[j])
                    time_at_max_velocity = time[np.argmax(data[j])]

                    # Add annotation for max velocity
                    ax.annotate(f'Max = {max_velocity:.2f}',
                            xy=(time_at_max_velocity, max_velocity),
                            xytext=(time_at_max_velocity + 3, max_velocity),
                            arrowprops=dict(facecolor='red', arrowstyle='->'),
                            fontsize=15,
                            color='red',horizontalalignment='left', verticalalignment='top')
                
                
            else:  # For Frequency Analysis
                ax.plot(freq[freq > 0], np.abs(data[j][freq > 0]), label=f'FFT ({axis.upper()}-axis)')
                ax.set_xlabel('Frequency (Hz)')

                # Additional lines to annotate dominant frequency and amplitude
                pos_freq = freq[freq > 0]
                pos_sp = np.abs(sp[freq > 0])
                
                dominant_frequency = pos_freq[np.argmax(pos_sp)]
                amplitude_at_dominant_frequency = np.max(pos_sp)
                
                ax.annotate(f'Dominant Frequency: {dominant_frequency:.2f} Hz\nAmplitude: {amplitude_at_dominant_frequency:.2f}', 
                xy=(dominant_frequency, amplitude_at_dominant_frequency), 
                xytext=(dominant_frequency + 3, amplitude_at_dominant_frequency),
                arrowprops=dict(facecolor='red', arrowstyle='->'),
                fontsize=15,
                color='red',horizontalalignment='left', verticalalignment='top')
                ax.plot(freq[freq > 0], np.abs(data[j][freq > 0]), label=f'FFT ({axis.upper()}-axis)')
                                
            ax.set_title(f'{titles[j]} ({axis.upper()}-axis)') #,y=1.08 for move subtitle upper side
            ax.set_ylabel(ylabels[j])
            
    
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_to_save):
        os.makedirs(folder_to_save)

    # Save the figure in the designated folder
    plt.savefig(os.path.join(folder_to_save, f"{full_title}.png"))
    #plt.tight_layout(pad=3.0)
    #plt.subplots_adjust(hspace=0.5)
    #plt.show()    

# Run the function on the dataset
# plot_movement_grav_HF_anno('jog_9/sub_7.csv', 'Plots')

### Now let's Train a machine learning model 
- ##### objective : to classify types of activity being performed.

- combine all data into 1 big dataframe
- add label to another column

In [ ]:
# folders = ['dws_1','dws_2','dws_11','jog_9','jog_16','sit_5','sit_13','std_6','std_14','ups_3','ups_4','ups_12','wlk_7','wlk_8','wlk_15']
folders = ['dws_1','dws_2','dws_11','jog_9','jog_16','sit_5','sit_13','std_6','std_14','ups_3','ups_4','ups_12','wlk_7','wlk_8','wlk_15']
all_data = [] # This list will store all the data


# ex) df = pd.read_csv('./Motion/jog_9/sub_7.csv')

for folder in folders:
    for i in range(1,24):
        file_path = f'./Motion/{folder}/sub_{i}.csv'
        df = pd.read_csv(file_path)
        df['activity'] = folder[:3]
        all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)

In [ ]:
df_all

## Experiment 
### To compare

In [ ]:
'''
dws_1
dws_2
dws_11
jog_9
jog_16
sit_5
sit_13
std_6
std_14
ups_3
ups_4
ups_12
wlk_7
wlk_8
wlk_15

'dws_1','dws_2','dws_11','jog_9','jog_16',
'''
folders = ['sit_5','sit_13','std_6','std_14','ups_3','ups_4','ups_12','wlk_7','wlk_8','wlk_15']

#plot_movement_grav('dws_1/sub_7.csv')
for folder in folders:
    for _ in range(1,24):
        plot_movement_grav_HF_anno(f'{folder}/sub_{_}.csv','Plots')


In [ ]:
plot_movement('jog_9/sub_1.csv')

In [ ]:


# Assuming df is your DataFrame and 'activity_label' is the column that has activity labels
# Add a kinetic energy column (Assuming velocity is already calculated and stored as 'velocity' column)
df['kinetic_energy'] = 0.5 * df['velocity'] ** 2

# Features and Labels
X = df[['kinetic_energy', 'velocity', 'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']]
y = df['activity_label']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Model Evaluation
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
for j in range(4):
    ax = axs[i, j]
    if j < 3:  # For Acceleration, Velocity, and Position
        ax.plot(time, data[j], label=f'{titles[j]} ({axis.upper()}-axis)')
        
        # Add average line for Velocity plot
        if titles[j] == 'Velocity':
            avg_velocity = np.mean(data[j])
            ax.axhline(y=avg_velocity, color='r', linestyle='--', label=f'Average Velocity = {avg_velocity:.2f}')

        ax.set_xlabel('Time (s)')
    # ... (rest of your code)
